## This notebook shows how to introduce human as user agent into conversation

In [45]:
from dataclasses import dataclass, field
from typing import List, Optional
import base64
from io import BytesIO
from autogen_core.base import MessageContext, AgentId, AgentInstantiationContext, TopicId
from autogen_core.components import DefaultTopicId, RoutedAgent, default_subscription, message_handler, TypeSubscription
from autogen_core.components.code_executor import CodeExecutor, extract_markdown_code_blocks
from autogen_core.components.models import (
    AssistantMessage,
    ChatCompletionClient,
    LLMMessage,
    SystemMessage,
    UserMessage,
)
import tempfile
from autogen_core.components.tool_agent import ToolAgent, tool_agent_caller_loop
from autogen_core.application import SingleThreadedAgentRuntime
from autogen_core.components.models import OpenAIChatCompletionClient
import random

from autogen_core.base import CancellationToken
from autogen_core.components.tools import FunctionTool, ToolSchema
from autogen_core.components._image import Image
from typing_extensions import Annotated
from pydantic import BaseModel
from PIL import Image as PILImage
import os
import uuid;

# Message Contract

Here we introduce two contracts
- TextMessage: the text message
- Conversation: the chat history

In [46]:
class TextMessage(BaseModel):
    text: str
    source: str

class Conversation(BaseModel):
    chat_history: List[TextMessage]

## UserAgent
We first define the UserAgent class, which represents the human user in the conversation.

The UserAgent handles Conversation in the following way. When it receives a conversation, it first prompts the user for a response.
If user says `TERMINATE`, the conversation is terminated. Otherwise, the user's response is added to the conversation and publish to agent runtime.

In [47]:
class UserAgent(RoutedAgent):
    def __init__(self, description: str, assistant_topic_type: str) -> None:
        super().__init__(description=description)
        self._assistant_topic_type = assistant_topic_type

    @message_handler
    async def handle_request_to_speak(self, message: Conversation, ctx: MessageContext) -> None:
        user_input = input("Enter your message, type 'TERMINATE' to terminate the task: ")
        print(f"### User: \n{user_input}")
        if user_input == "TERMINATE": # end the conversation
            return
        
        message.chat_history.append(TextMessage(text=user_input, source=self.type))
        await self.publish_message(
            message,
            DefaultTopicId(type=self._assistant_topic_type),
        )

## Assistant

The Assistant class is the agent that interacts with the user. It receives the conversation from the UserAgent, then uses LLM to generate a response. The response is then added to the conversation and published to the UserAgent.

In [48]:
class Assistant(RoutedAgent):
    def __init__(self,
                 model_client: ChatCompletionClient,
                 system_message: str = "You are a helpful AI assistant.",
                 user_topic_type: str = "user") -> None:
        super().__init__("An assistant agent.")
        self._model_client = model_client
        self._system_message = system_message
        self._user_topic_type = user_topic_type

    @message_handler
    async def handle_message(self, message: Conversation, ctx: MessageContext) -> None:
        chat_history: List[LLMMessage] = [SystemMessage(content=self._system_message)]
        for msg in message.chat_history:
            if msg.source == self.type:
                chat_history.append(AssistantMessage(content=msg.text, source="assistant"))
            elif msg.source == self._user_topic_type:
                chat_history.append(UserMessage(content=msg.text, source="user"))
            else:
                raise ValueError(f"Unknown message source: {msg.source}")
            
        result = await self._model_client.create(chat_history)
        print(f"\n{'-'*80}\nAssistant:\n{result.content}")
        message.chat_history.append(TextMessage(text=result.content, source=self.type))
        await self.publish_message(message, topic_id=DefaultTopicId(type=self._user_topic_type))

In [50]:
runtime = SingleThreadedAgentRuntime()
user_agent_type = await UserAgent.register(
    runtime,
    type="user",
    factory=lambda: UserAgent(
        description="A user agent that generates messages for the assistant agent.",
        assistant_topic_type="assistant"
    ),
)

assistant_agent_type = await Assistant.register(
    runtime,
    type="assistant",
    factory=lambda: Assistant(
        model_client=OpenAIChatCompletionClient(
            api_key=os.environ.get("OPENAI_API_KEY"),
            model="gpt-4o-mini"
        ),
        user_topic_type="user"
    ),
)

await runtime.add_subscription(
    TypeSubscription("user", user_agent_type.type))
await runtime.add_subscription(
    TypeSubscription("assistant", assistant_agent_type.type))

runtime.start()
session_id = str(uuid.uuid4())
msg = Conversation(chat_history=[])
await runtime.publish_message(
    msg,
    TopicId(type="assistant", source=session_id),
)
await runtime.stop_when_idle()



--------------------------------------------------------------------------------
Assistant:
How can I assist you today?
### User: 
I am Rich Lander, tell me a Joke

--------------------------------------------------------------------------------
Assistant:
Sure, Rich! Here’s a joke for you:

Why did the scarecrow win an award?

Because he was outstanding in his field!
### User: 
Sumarize the conversation

--------------------------------------------------------------------------------
Assistant:
Rich Lander asked for a joke, and I provided one about a scarecrow winning an award for being outstanding in his field.
### User: 
Who are you talking to

--------------------------------------------------------------------------------
Assistant:
I'm talking to you, Rich Lander! How can I assist you further?
### User: 
TERMINATE


## What we learn
- How to introduce human as user agent into conversation